referensi https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/

In [1]:
# # instal library mlxtend
# %pip install mlxtend

In [2]:
# Import Library
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.feature_selection import mutual_info_regression,SelectKBest 
from sklearn.ensemble import RandomForestRegressor
from skfeature.function.similarity_based import fisher_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

# Seleksi Fitur Fisher Score (k=300)

In [4]:
%%time

#cari ranking setiap fitur berdasarkan fisher score
fisher_score = fisher_score.fisher_score(x_train.to_numpy(), y_train.to_numpy())

fisher_score

/home/jupyter-17523142/.local/lib/python3.7/site-packages/skfeature/function/similarity_based/fisher_score.py:59: RuntimeWarning: divide by zero encountered in true_divide
  score = 1.0 / lap_score - 1


CPU times: user 6.48 s, sys: 139 ms, total: 6.62 s
Wall time: 458 ms


array([ 778, 1022, 1031, ..., 1216, 1218,    0])

In [5]:
# Ascending
# bikin dataframe buat nyimpan fitur beserta rankingnya
fisher_ranking = pd.DataFrame(fisher_score)
features = pd.DataFrame(x_train.columns)
fisher_ranking = pd.concat([features,fisher_ranking],axis=1)

# Assign the column name
fisher_ranking.columns = ["Features","Fisher_Ranking"]

# Print features score
print(fisher_ranking.nsmallest(300,columns="Fisher_Ranking"))

     Features  Fisher_Ranking
1556   2500.2               0
260    1111.3               1
1188   1845.4               2
1406   2184.3               3
1189   1846.7               4
...       ...             ...
1493     2357             295
802    1447.6             296
801    1446.8             297
800      1446             298
798    1444.4             299

[300 rows x 2 columns]


# Seleksi Fitur Backward Elimination dengan Linear Regression

In [6]:
x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:300, 0]]
x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:300, 0]]

In [7]:
x_train_selected

,2500.2,1111.3,1845.4,2184.3,1846.7,1852,1853.3,1857.3,1863.9,1838.8,...,1451.7,1450.9,1450.1,2359.2,1448.5,2357,1447.6,1446.8,1446,1444.4
16,1.428625,0.438341,0.907775,1.097736,0.908989,0.921439,0.924654,0.938583,0.971858,0.899052,...,1.108341,1.108181,1.108373,1.256366,1.108033,1.255772,1.107926,1.107721,1.107372,1.106641
51,1.611099,0.391662,1.065262,1.307243,1.067778,1.085994,1.091106,1.112092,1.160458,1.052971,...,1.356351,1.356063,1.356032,1.467247,1.356365,1.465697,1.356000,1.355492,1.354792,1.354059
183,1.533183,0.497537,1.140941,1.305443,1.142446,1.155736,1.159088,1.173587,1.206242,1.131616,...,1.325683,1.325006,1.324760,1.429789,1.324391,1.429389,1.324223,1.323797,1.323525,1.323314
145,1.428271,0.501907,1.090478,1.247814,1.091900,1.103762,1.106702,1.119947,1.149629,1.082253,...,1.253276,1.253272,1.253152,1.347394,1.253142,1.346965,1.252922,1.252735,1.252549,1.251983
40,1.581424,0.339756,0.950444,1.214149,0.952297,0.968828,0.973166,0.992849,1.039250,0.939128,...,1.248552,1.248345,1.248367,1.424007,1.248150,1.422121,1.247726,1.247627,1.247304,1.246680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,1.515658,0.476264,1.048784,1.238116,1.050575,1.064826,1.068715,1.084751,1.121644,1.038848,...,1.272931,1.272609,1.272558,1.378025,1.272289,1.376241,1.272051,1.271575,1.271002,1.270537
67,1.516333,0.407174,1.014395,1.228384,1.016328,1.032534,1.036883,1.054946,1.096734,1.003201,...,1.258628,1.258800,1.259068,1.378647,1.258452,1.377451,1.258343,1.258136,1.257725,1.257308
117,1.609544,0.426150,1.178860,1.404545,1.181145,1.199104,1.204185,1.224251,1.269758,1.166576,...,1.428554,1.428430,1.428178,1.524956,1.428004,1.523609,1.427889,1.427436,1.427210,1.427226
47,1.591581,0.386672,1.062860,1.298707,1.065343,1.083832,1.089076,1.109692,1.158657,1.050109,...,1.354656,1.354440,1.354414,1.454723,1.353886,1.452482,1.353804,1.353888,1.353531,1.353050


In [8]:
# calling the Linear Regression model
lr_model =  LinearRegression()
sfs = sfs(lr_model, k_features=60, forward=False, verbose=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

In [9]:
%%time

sfs = sfs.fit(x_train_selected, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 292 out of 300 | elapsed:    7.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    7.8s finished

[2022-07-14 23:29:24] Features: 299/60 -- score: -3509628.81994543[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 290 out of 299 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 299 out of 299 | elapsed:    1.5s finished

[2022-07-14 23:29:25] Features: 298/60 -- score: -2486034.5419186438[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 289 out of 298 | elapsed:    1.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 298 out of 298 | elapsed:    1.7s 

CPU times: user 26min 8s, sys: 28.3 s, total: 26min 36s
Wall time: 26min 20s


[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    4.1s finished

[2022-07-14 23:55:36] Features: 60/60 -- score: -15.218017369198218

In [10]:
feat_names = list(sfs.k_feature_names_)
print("Fitur yang diambil adalah: ")
print(feat_names)
len(feat_names)

Fitur yang diambil adalah: 
['2500.2', '1857.3', '1838.8', '1837.5', '1905', '1896.7', '1913.5', '1885.6', '1723.9', '1726.2', '1732', '1798', '1768.6', '1785.6', '2102.8', '2000.6', '1496.1', '1478.2', '1538.7', '1425.4', '1455.8', '1442', '1669.5', '1671.7', '1711.4', '1612.4', '1623.5', '1600.5', '2350.6', '1595.5', '1592.6', '1591.6', '1586.7', '1621.5', '1620.5', '2414.1', '1615.4', '2411.8', '1561.9', '1557.2', '1551.6', '1580.9', '1578', '1567.6', '1679.2', '1678.1', '1674.9', '1672.7', '1645.1', '1643.1', '1637.9', '1632.7', '1648.3', '1668.4', '1666.3', '1665.2', '1664.1', '2376.5', '1652.5', '2365.6']


60

# Prediksi Linear Regression

In [11]:
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [12]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [60]

for nfeat in n_feat:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Linear Regression
        lr_model = LinearRegression()
        
        #Train the model using the training sets
        lr_model.fit(x_train_selected, y_train)
        y_pred_lr = lr_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
        print("RMSE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
        print("MAE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
        print("R2 model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 60 fitur: 70.23
RMSE model Linear Regression data Train dengan 60 fitur:8.37
MAE model Linear Regression data Train dengan 60 fitur:6.58
R2 model Linear Regression data Train dengan 60 fitur:0.63
----------------------------
MSE model Linear data Test dengan 60 fitur:633.31
RMSE model Linear data Test dengan 60 fitur:25.17
MAE model Linear data Test dengan 60 fitur:14.65
R2 model Linear data Test dengan 60 fitur:2.67
CPU times: user 5.01 s, sys: 142 ms, total: 5.16 s
Wall time: 217 ms


# Prediksi Random Forest Regressor

## 100 Trees

In [13]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [60]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 60 fitur: 38.48
RMSE model Linear Regression data Train dengan 60 fitur:6.2
MAE model Linear Regression data Train dengan 60 fitur:4.72
R2 model Linear Regression data Train dengan 60 fitur:0.8
----------------------------
MSE model Linear data Test dengan 60 fitur:177.87
RMSE model Linear data Test dengan 60 fitur:13.34
MAE model Linear data Test dengan 60 fitur:10.42
R2 model Linear data Test dengan 60 fitur:0.03
CPU times: user 6.14 s, sys: 107 ms, total: 6.25 s
Wall time: 2.5 s


## 150 Trees

In [14]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [60]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 60 fitur: 38.16
RMSE model Linear Regression data Train dengan 60 fitur:6.17
MAE model Linear Regression data Train dengan 60 fitur:4.72
R2 model Linear Regression data Train dengan 60 fitur:0.8
----------------------------
MSE model Linear data Test dengan 60 fitur:179.85
RMSE model Linear data Test dengan 60 fitur:13.41
MAE model Linear data Test dengan 60 fitur:10.47
R2 model Linear data Test dengan 60 fitur:0.04
CPU times: user 3.59 s, sys: 27.9 ms, total: 3.62 s
Wall time: 3.62 s


## 200 Trees

In [15]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [60]
n_trees = [200]
for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nlargest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 60 fitur: 39.24
RMSE model Linear Regression data Train dengan 60 fitur:6.26
MAE model Linear Regression data Train dengan 60 fitur:4.78
R2 model Linear Regression data Train dengan 60 fitur:0.8
----------------------------
MSE model Linear data Test dengan 60 fitur:174.45
RMSE model Linear data Test dengan 60 fitur:13.21
MAE model Linear data Test dengan 60 fitur:10.49
R2 model Linear data Test dengan 60 fitur:0.01
CPU times: user 4.27 s, sys: 59.8 ms, total: 4.33 s
Wall time: 4.32 s
